## Trees

We'll cover the following topics in this notebook:
- Tree operations: `map`, scanning, `fold`
- Generating trees
- Option type and its usage
- Using datatypes to express a syntax tree
- Implement an interpreter via pattern matching

In [ ]:
type 'a tree = Lf | Br of 'a * 'a tree * 'a tree

All data types have their corresponding versions of `map` and `fold`. Mapping a function `f` over a tree is simply distributing this function over all children nodes of the tree.

In [ ]:
let rec map_tree f = function 
  | Lf -> Lf
  | Br (a, t1, t2) -> Br (f a, map_tree f t1, map_tree f t2)

We can also define tree-scanning functions `mem`, `forall`, and `exist`, just like we did for lists.

In [ ]:
let rec mem x = function
  | Lf -> false
  | Br (a, t1, t2) -> a = x || mem x t1 || mem x t2

let rec forall f = function
  | Lf -> true
  | Br (a, t1, t2) -> f a && forall f t1 && forall f t2

let rec exist f = function
  | Lf -> false
  | Br (a, t1, t2) -> f a || exist f t1 || exist f t2

To better understand `fold`, we look at the special cases, `count` and `sum`.

In [ ]:
let rec count_tree = function 
  | Lf -> 0
  | Br (a, t1, t2) -> 1 + (count_tree t1) + (count_tree t2)

In [ ]:
let rec sum_tree = function 
  | Lf -> 0
  | Br (a, t1, t2) -> a + (sum_tree t1) + (sum_tree t2)

Now, we can generalize the pattern here: at the base case, we return some initial value; at recursive case, we use a function `f : 'a -> 'b -> 'b -> 'b` to combine the values of the current node's element and the outcomes from the recursive calls on the two subtrees.

In [ ]:
let rec fold_tree init f = function  
  | Lf -> init
  | Br (a, t1, t2) -> f a (fold_tree init f t1) (fold_tree init f t2)

Using `fold_tree` to write the `depth` function:

In [ ]:
let rec depth = function  
  | Lf -> 0
  | Br (a, t1, t2) -> 1 + max (depth t1) (depth t2)

In [ ]:
let depth tr = fold_tree 0 (fun a r1 r2 -> 1 + max r1 r2)

### Generating trees

We learned how to represent trees and how to map and fold over a tree. Now we look at ways to generate int-valued trees with certain patterns.

- `ftree : int -> int -> int tree` generates a balanced tree with `k` on the top and `n` levels. For example, `ftree 1 3` gives a tree like `1 [2 [4 5]] [3 [6 7]]`.
- `farr : int -> int tree` generates a functional array of size $2^n - 1$, where $n$ is the function argument. For example, `farr 3` gives a tree like `1 [2 [3 6]] [4 [5 7]]`

Key to generating these trees is to find the invariant between a node and its two children. For trees generated by `ftree`, the values of a node, its left child, and its right child satisfies $(n, l, r) = (k, 2k, 2k + 1)$. So, we give a recursive definition like the following:

In [ ]:
let rec ftree k n =
	if n = 0 then Lf
	else Br (k, ftree (2 * k) (n - 1), ftree (2 * k + 1) (n - 1))

Similarly, for trees generated by `farr`, a node and its two children satisfies $(n, l, r) = (k, k + s, k + 2s)$, where $s = 2^l$, $l$ is the node's level.

In [ ]:
let farr n = 
  let rec farr' n k s =
    if n = 0 then Lf
    else Br (k, farr' (n - 1) (k + s) (s * 2), farr' (n - 1) (k + s * 2) (s * 2))
  in
    farr' n 1 1

### Option types

We can define an option type to represent values that could be an error.

In [ ]:
type 'a option = None | Some of 'a

The option type is perfect for operations that might result in error, but we don't want to throw exceptions. For example, we can define `find` for list with exceptions as the follows.

In [ ]:
let rec find f = function
  | [] -> raise Not_found
  | x :: xs -> if f x then x else find f xs

This definition has type `('a -> bool) -> 'a list -> 'a`. At the case of empty list, we can't find an element of type `'a`, so we have to throw an exception. Alternatively, we can make its type be `('a -> bool) -> 'a list -> 'a option`, and return `None` at the base case.

In [ ]:
let rec find_opt f = function 
  | [] -> None
  | x :: xs -> if f x then Some x else find f xs

It's very common to define a bind operation for option type. Given a function of type `'a -> 'b option` and some `'a option`, if the option value is `Some x`, then apply the function to `x`, otherwise, we propogate the `None` forward. This is usually written as an infix symbol (>>), so that we can bind many operations together like a chain: `Some x >> f >> g >> ...`.

In [ ]:
let (>>) x f = match x with
  | None -> None 
  | Some x -> f x

### Calculator

An arithmetic expression, like `2 + 3 * 5`, can be parsed as `ADD 2 (MULT 3 5)`, which has a tree-like structure. The leaf nodes contain an integer, and the branch nodes are annotated with different operators like `ADD` and `MULT`. We call these structures abstract syntax trees, and we can describe them easily with a datatype similar to trees.

In [ ]:
data expr = 
  | INT of int
  | ADD of int * int
  | SUB of int * int
  | MULT of int * int
  | DIV of int * int

Then, we can write an recursive function to calculate an `expr`, which we call `interp`. This function should take an `expr` as input and returns `int option` as output, since it might go wrong if we divide by zero. That makes our coding slightly more complicated, since we need to think about how to add/subtract/multiply a `None` and a `Some`.

In [ ]:
let bop f x y = match (x , y) with
  | None , _ | _ , None -> None 
  | Some x , Some y -> Some (f x y)

let div x y = match (x , y) with
  | None , _ | _ , None | _ , Some 0 -> None 
  | Some x , Some y -> x / y

let rec interp = function
  | INT n -> Some n
  | ADD (e1 , e2) -> bop (+) (interp e1) (interp e2)
  | SUB (e1 , e2) -> bop (-) (interp e1) (interp e2)
  | MULT (e1 , e2) -> bop (*) (interp e1) (interp e2)
  | DIV (e1 , e2) -> div (interp e1) (interp e2)

We can now try to compute some expressions!

In [ ]:
ADD(INT 2, MULT(INT 3, INT 5))

In [ ]:
DIV(ADD(INT 1, INT 4), SUB(INT 3, INT 3))

In [ ]:
ADD(INT 1, DIV(INT 4, INT 0))